In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np

class Node:
    def __init__(self, x=None, y=None, hue=None, frequency=None):
        self.x = x
        self.y = y
        self.hue = hue
        self.frequency = frequency

# Define the area dimensions
area_width = 379
area_height = 379
grid_size = 1  # Set grid size to 1 to cover every possible (x, y) coordinate

# Generate the set of 8 nodes evenly spaced on the hue spectrum and mapped to the audible spectrum
hue_step = 360 / 8
frequency_range = (600, 600 * 8)  # Audible frequency range in Hz
frequency_step = 600  # Logarithmic spacing
nodes = [Node(hue=i * hue_step, frequency=frequency_range[0] + i * frequency_step) for i in range(8)]

# Create a grid of nodes covering the entire area
grid = [[random.choice(nodes) for _ in range(area_width // grid_size)] for _ in range(area_height // grid_size)]

# Flatten the grid into a list of nodes
all_nodes = [node for row in grid for node in row]

# Assign positions to nodes based on grid cell coordinates
for i, node in enumerate(all_nodes):
    node.x = (i % (area_width // grid_size)) * grid_size + random.uniform(0, grid_size)
    node.y = (i // (area_width // grid_size)) * grid_size + random.uniform(0, grid_size)

# Create lists of x and y coordinates, hues, and frequencies for plotting
x_coords = [node.x for node in all_nodes]
y_coords = [node.y for node in all_nodes]
hues = [node.hue for node in all_nodes]
frequencies = [node.frequency for node in all_nodes]

# Plot the nodes as a scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(x_coords, y_coords, c=hues, cmap='hsv', alpha=0.8)
plt.colorbar(label='Hue')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('Node Distribution with Hues and Frequencies')
plt.grid(True)
plt.show()


In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np

class Node:
    def __init__(self, x=None, y=None, hue=None, frequency=None, layer=None):
        self.x = x
        self.y = y
        self.hue = hue
        self.frequency = frequency
        self.layer = layer
    
    def copy(self):
        return Node(self.x, self.y, self.hue, self.frequency, self.layer)
    

area_width = 379
area_height = 379
grid_size = 10  # Adjust the grid size as needed

# Generate the set of 8 nodes evenly spaced on the hue spectrum and mapped to the audible spectrum
hue_step = 360 / 8
frequency_range = (600, 600 * 8)  # Audible frequency range in Hz
frequency_step = 600  # Logarithmic spacing
nodes = [Node(hue=i * hue_step, frequency=frequency_range[0] + i * frequency_step) for i in range(8)]

for xx in range(0,378):
    for yy in range(0,378):
        node = random.choice(nodes).copy()
        node.x = xx
        node.y = yy
        node.layer = 0









In [ ]:
from deap import base, creator, tools, algorithms
import random

# Define individual and population
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("random_bit", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, list, toolbox.random_bit, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Evaluation function
def evaluate(individual):
    """Evaluate the fitness of an individual based on its value."""
    # Example fitness evaluation: simply return the value of the bit
    return individual[0],

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Evolutionary algorithm
# Evolutionary algorithm
def evolve(population, n_generations):
    for gen in range(n_generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
        fits = toolbox.map(toolbox.evaluate, offspring)
        for fit, ind in zip(fits, offspring):
            ind.fitness.values = fit
        population = toolbox.select(offspring, k=len(population))
    return population

# Example usage
population = toolbox.population(n=100)
final_population = evolve(population, n_generations=50)


In [ ]:
from deap import base, creator, tools
import random
import numpy as np

# Game of Life utilities
def count_neighbors(grid, x, y):
    """Count the number of live neighbors around cell (x, y) in the grid."""
    count = 0
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx == 0 and dy == 0:
                continue
            if 0 <= x + dx < len(grid) and 0 <= y + dy < len(grid[0]) and grid[x + dx][y + dy] == 1:
                count += 1
    return count

def evaluate(individual):
    """Evaluate the fitness of an individual based on its configuration in the Game of Life."""
    grid = np.array(individual).reshape(10, 10)
    new_grid = np.zeros_like(grid)
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            live_neighbors = count_neighbors(grid, i, j)
            if grid[i][j] == 1:  # Live cell
                if live_neighbors < 1:
                    new_grid[i][j] = 0  # Dies due to underpopulation
                elif live_neighbors == 1:
                    new_grid[i][j] = 1  # Survives and produces offspring
                elif live_neighbors == 2 or live_neighbors == 3:
                    new_grid[i][j] = 1  # Survives
                else:
                    new_grid[i][j] = 0  # Dies due to overpopulation
            else:  # Dead cell
                if live_neighbors == 3:
                    new_grid[i][j] = 1  # Reproduction
    return np.sum(new_grid),

# DEAP setup
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("random_bit", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, list, toolbox.random_bit, n=100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Evolutionary algorithm
def evolve(population, n_generations):
    for gen in range(n_generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
        fits = toolbox.map(toolbox.evaluate, offspring)
        for fit, ind in zip(fits, offspring):
            ind.fitness.values = fit
        population = toolbox.select(offspring, k=len(population))
    return population

# Example usage
population = toolbox.population(n=100)
final_population = evolve(population, n_generations=50)

In [ ]:
import numpy as np
import pandas as pd

stability_table = np.array([
[-1,1,	2,	3,	4,	5,	6,	7,	8],
[1,	1,	0,	0,	0,	0,	0,	0,	0],
[2,	1,	1,	1,	1,	1,	1,	1,	1],
[3,	1,	0,	1,	1,	1,	1,	0,	0],
[4,	1,	0,	0,	1,	1,	1,	0,	0],
[5,	1,	0,	0,	0,	1,	0,	0,	0],
[6,	1,	0,	0,	0,	1,	1,	0,	0],
[7,	1,	0,	0,	0,	1,	1,	1,	1],
[8,	1,	0,	0,	1,	1,	1,	0,	1]
])


class Node:
    def __init__(self, top_type, bottom_type, stability, magnitude, pressure):
        self.top_type = top_type
        self.bottom_type = bottom_type
        self.stability = stability
        self.magnitude = magnitude
        self.pressure = pressure
    
    def __str__(self):
        return f"{self.stability}{self.top_type}{self.bottom_type};{self.magnitude};{self.pressure}"

# Function to generate a random individual (pairing)
def random_individual():
    # Choose a random index for "top" and "bottom" node types
    top_index = np.random.randint(1, 9)
    bottom_index = np.random.randint(1, 9)
    # Determine stability based on the stability table
    top_type = stability_table[top_index][0]
    bottom_type = stability_table[0][bottom_index]
    stability = stability_table[top_index][bottom_index]
    # Choose constant magnitude and pressure values
    magnitude = 1 
    pressure = 0
    # Create a Node object with the determined properties
    return Node(top_type, bottom_type, stability, magnitude, pressure)

# Function to initialize the grid with random pairings
def initialize_grid(size):
    grid = np.array([[random_individual() for _ in range(size)] for _ in range(size)])  # Generate random pairings for each cell
    return grid

# Initialize the grid
grid_size = 30
grid = initialize_grid(grid_size)

# Create DataFrame from the grid
df = pd.DataFrame(grid)

# Print the DataFrame
print(df)


current

In [36]:
import numpy as np

# Stability table with node types
stability_table = np.array([
    [-1, 1, 2, 3, 4, 5, 6, 7, 8],
    [1, 1, 0, 0, 0, 0, 0, 0, 0],
    [2, 1, 1, 1, 1, 1, 1, 1, 1],
    [3, 1, 0, 1, 1, 1, 1, 0, 0],
    [4, 1, 0, 0, 1, 1, 1, 0, 0],
    [5, 1, 0, 0, 0, 1, 0, 0, 0],
    [6, 1, 0, 0, 0, 1, 1, 0, 0],
    [7, 1, 0, 0, 0, 1, 1, 1, 1],
    [8, 1, 0, 0, 1, 1, 1, 0, 1]
])

class Node:
    def __init__(self, x, y, top_type, bottom_type, stability, magnitude, pressure):
        self.x = x
        self.y = y
        self.top_type = top_type
        self.bottom_type = bottom_type
        self.stability = stability
        self.magnitude = magnitude
        self.pressure = pressure
    
    def __str__(self):
        return f"{self.stability}{self.top_type}{self.bottom_type};{self.magnitude};{self.pressure};{self.x}:{self.y}{self.layer}"

# Function to generate a random individual (pairing)
def random_individual():
    # Choose a random index for "top" and "bottom" node types
    top_index = np.random.randint(1, 9)
    bottom_index = np.random.randint(1, 9)
    # Determine stability based on the stability table
    top_type = stability_table[top_index][0]
    bottom_type = stability_table[0][bottom_index]
    stability = stability_table[top_index][bottom_index]
    # Choose constant magnitude and pressure values
    magnitude = 1 
    pressure = 0
    # Create a Node object with the determined properties
    return Node(top_type, bottom_type, stability, magnitude, pressure)

# Function to initialize the grid with random pairings
def initialize_grid(size):
    grid = np.array([[random_individual() for x in range(size)] for y in range(size)])  # Generate random pairings for each cell
    return grid

# Initialize the grid
grid_size = 30
grid = initialize_grid(grid_size)

# Print the grid
print(grid)

[[<__main__.Node object at 0x7a37d12fa950>


In [37]:
def simulate_single_turn(grid):
    updated_grid = np.copy(grid)  # Create a copy of the grid to store the updated state
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            pair = grid[i, j]  # Get the pair at the current grid position
            top_stable = pair.stability == 1  # Check if the pair is stable
            
            # Rule 1: All pairs gain 0.1 magnitude per turn
            pair.magnitude += 0.1
            
            # Rule 2: An empty neighbor creates a .9 magnitude unstable result
            # Note: We'll handle this rule when iterating over neighboring pairs
            
            # Rule 5: An unstable pair spends 1 magnitude to flip to stable (0->1)
            if not top_stable:
                if pair.magnitude >= 1:
                    pair.stability = 1
                    pair.magnitude -= 1
            
            # Rule 6: Lower of an unstable pair exerts pressure on upper nodes vertically
            if not top_stable:
                # Check neighboring upper nodes (up and down)
                for k in [-1, 1]:
                    if 0 <= i + k < grid.shape[0]:
                        neighbor_pair = grid[i + k, j]
                        if neighbor_pair.stability == 1:
                            if neighbor_pair.magnitude > pair.pressure:
                                pair.pressure += neighbor_pair.magnitude
            
            # Rule 8: Unstable pressure is exerted by checking stability and magnitude with neighboring upper
            if not top_stable:
                # Check neighboring upper nodes (up and down)
                for k in [-1, 1]:
                    if 0 <= i + k < grid.shape[0]:
                        neighbor_pair = grid[i + k, j]
                        if not neighbor_pair.stability and neighbor_pair.magnitude >= 0.9:
                            pair.magnitude += 0.9
                            neighbor_pair.magnitude -= 0.9
            
            # Rule 9: A stable pair that flips releases all magnitude > 0.1 to neighbors (even split)
            if top_stable and pair.magnitude > 0.1:
                remaining_magnitude = pair.magnitude - 0.1
                num_neighbors = 0
                total_pressure = 0
                # Check neighboring upper nodes (up and down)
                for k in [-1, 1]:
                    if 0 <= i + k < grid.shape[0]:
                        neighbor_pair = grid[i + k, j]
                        if neighbor_pair.stability == 1:
                            num_neighbors += 1
                            total_pressure += neighbor_pair.pressure
                if num_neighbors > 0:
                    pressure_per_neighbor = total_pressure / num_neighbors
                    for k in [-1, 1]:
                        if 0 <= i + k < grid.shape[0]:
                            neighbor_pair = grid[i + k, j]
                            if neighbor_pair.stability == 1:
                                neighbor_pair.magnitude += remaining_magnitude / num_neighbors
                                neighbor_pair.pressure += pressure_per_neighbor
            
            # Rule 10: An unstable pair that cannot flip releases 0.1 magnitude to neighbors (even split)
            if not top_stable and pair.magnitude < 1:
                num_neighbors = 0
                total_pressure = 0
                # Check neighboring upper nodes (up and down)
                for k in [-1, 1]:
                    if 0 <= i + k < grid.shape[0]:
                        neighbor_pair = grid[i + k, j]
                        if neighbor_pair.stability == 1:
                            num_neighbors += 1
                            total_pressure += neighbor_pair.pressure
                if num_neighbors > 0:
                    pressure_per_neighbor = total_pressure / num_neighbors
                    for k in [-1, 1]:
                        if 0 <= i + k < grid.shape[0]:
                            neighbor_pair = grid[i + k, j]
                            if neighbor_pair.stability == 1:
                                neighbor_pair.magnitude += 0.1 / num_neighbors
                                neighbor_pair.pressure += pressure_per_neighbor
            
            # Reset pressure for the next turn
            pair.pressure = 0
            
            # Update the pair in the updated grid
            updated_grid[i, j] = pair
    
    return updated_grid


In [38]:
# Function to run the simulation for a specified number of turns
def run_simulation(grid, num_turns):
    print("Initial grid:")
    print(pd.DataFrame(grid))
    print("\n")
    
    for turn in range(num_turns):
        print(f"Turn {turn + 1}:")
        grid = simulate_single_turn(grid)
        print(pd.DataFrame(grid))
        print("\n")

In [ ]:
# Run the simulation for 5 turns
run_simulation(grid, num_turns=5)

to output

In [ ]:
import matplotlib.pyplot as plt

# Function to extract stability information from Node objects
def extract_stability(grid):
    stability_grid = np.zeros_like(grid, dtype=float)
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            stability_grid[i, j] = grid[i, j].stability
    return stability_grid

# Modified function to visualize the grid as a heatmap
def visualize_grid(grid, turn):
    stability_grid = extract_stability(grid)
    plt.figure(figsize=(8, 6))
    plt.imshow(stability_grid, cmap='cool', vmin=0, vmax=1)
    plt.colorbar(label='Stability')
    plt.title(f'Grid at Turn {turn}')
    plt.xlabel('Column')
    plt.ylabel('Row')
    plt.show()
    
# Function to run the simulation for a specified number of turns with visualization
def run_simulation_with_visualization(grid, num_turns):
    visualize_grid(grid, turn=0)  # Visualize initial grid
    
    for turn in range(num_turns):
        grid = simulate_single_turn(grid)
        visualize_grid(grid, turn=turn + 1)  # Visualize grid after each turn

# Run the simulation for 5 turns with visualization
run_simulation_with_visualization(grid, num_turns=5)

to file

In [39]:
import matplotlib.pyplot as plt
import os

# Function to extract stability information from Node objects
def extract_stability(grid):
    stability_grid = np.zeros_like(grid, dtype=float)
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            stability_grid[i, j] = grid[i, j].stability
    return stability_grid

# Directory to save the PNG files
output_dir = 'output_plots'

# Function to visualize the grid as a heatmap and save it as a PNG file
def visualize_grid(grid, turn):
    stability_grid = extract_stability(grid)
    plt.figure(figsize=(8, 6))
    plt.imshow(stability_grid, cmap='cool', vmin=0, vmax=1)
    plt.colorbar(label='Stability')
    plt.title(f'Grid at Turn {turn}')
    plt.xlabel('Column')
    plt.ylabel('Row')
    
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Save the plot as a PNG file with a numerical label
    plt.savefig(os.path.join(output_dir, f'grid_{turn:03d}.png'))
    plt.close()
    
# Function to run the simulation for a specified number of turns with visualization
def run_simulation_with_visualization(grid, num_turns):
    visualize_grid(grid, turn=0)  # Visualize initial grid
    
    for turn in range(num_turns):
        grid = simulate_single_turn(grid)
        visualize_grid(grid, turn=turn + 1)  # Visualize grid after each turn

# Run the simulation for 5 turns with visualization
run_simulation_with_visualization(grid, num_turns=5)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7a382f5a0510>>
Traceback (most recent call last):
  File "/home/zampinojosh/src/twc/evtwc/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
